In [56]:
# import necessary libraries
from datetime import date, datetime, timedelta
import mysql.connector
from mysql.connector import errorcode
import pandas as pd

DB_NAME = 'timperio'
USERNAME = 'root'
PASSWORD = 'password'
HOST = 'localhost'

# connect to the database
try:
    connection = mysql.connector.connect(host=HOST, user=USERNAME, password=PASSWORD, database=DB_NAME)
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your username/password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
        
cursor = connection.cursor()

In [57]:
# Load the data from Excel and convert the date format
df_sales = pd.read_csv('CSV_Sales_Data_updated_Sep 2023.csv')
# df_sales['Sale Date'] = pd.to_datetime(df_sales['Sale Date'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
df_sales = df_sales.replace({float("NaN"): None})

In [58]:
# Insert the data into the database
# Have to insert data in this order: Customer, Product, Sale

# Insert data into Customer table
try:
    for index, row in df_sales.iterrows():
        sql = """
        INSERT INTO customers (customer_id, zipcode) VALUES (%s, COALESCE(%s, 0))
        ON DUPLICATE KEY UPDATE 
            `zipcode` = CASE
                WHEN VALUES(zipcode) IS NOT NULL THEN VALUES(zipcode)
                ELSE zipcode
            END;
        """
        
        values = (
            row['Customer ID'],
            row['ZipCode']
        )
        cursor.execute(sql, values)
        connection.commit()
except mysql.connector.Error as err:
    print(err)

In [59]:
def insert_into_sequence(table_name, next_id):
    try:
        cursor.execute("INSERT IGNORE INTO sequence (table_name, next_id) VALUES (%s, %s)", (table_name, next_id))
        connection.commit()  # Commit transaction
    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Call the function
insert_into_sequence('product_id', 0)

In [60]:
# Insert data into Product table
try:
    for index, row in df_sales.iterrows():
        # Check if there is a product with the same name and variant
        cursor.execute("SELECT product_id FROM products WHERE product_name = %s AND product_variant = %s", (row['Product'], row['Variant']))
        product = cursor.fetchone()
        print(product)
        
        if product == None: # if there is no product with the same name and variant
            print("No product found")
            # Lock the sequence row for safe access
            cursor.execute("SELECT next_id FROM sequence WHERE table_name = 'product_id' FOR UPDATE")
            current_id = cursor.fetchone()[0]
            
            # Increment the ID
            new_id = current_id + 1
            cursor.execute("UPDATE sequence SET next_id = %s WHERE table_name = 'product_id'", (new_id,))
            
            # Insert the new product
            sql = """
            INSERT INTO products (product_id, product_name, product_variant, product_price) VALUES (%s, %s, %s, %s)
            """
            # ON DUPLICATE KEY UPDATE `product_price` = VALUES(product_price);
            # """
        
            values = (
                new_id,
                row['Product'],
                row['Variant'],
                row['Price']
            )
            cursor.execute(sql, values)
            connection.commit()
except mysql.connector.Error as err:
    connection.rollback()
    print(err)

None
No product found
None
No product found
None
No product found
None
No product found
(1,)
None
No product found
None
No product found
(1,)
(1,)
(5,)
(6,)
(4,)
(3,)
None
No product found
None
No product found
(8,)
(7,)
(1,)
(3,)
(8,)
(8,)
None
No product found
(8,)
(8,)
None
No product found
(2,)
None
No product found
(1,)
(5,)
(6,)
(7,)
(3,)
(4,)
(8,)
None
No product found
(10,)
None
No product found
(9,)
(12,)
(9,)
(1,)
(6,)
(4,)
(8,)
(4,)
(7,)
(8,)
(10,)
None
No product found
(8,)
(12,)
(8,)
(2,)
(12,)
(8,)
(1,)
(4,)
(8,)
(10,)
(10,)
(4,)
(8,)
(8,)
(11,)
(13,)
(4,)
(8,)
(4,)
(13,)
(1,)
(5,)
(6,)
(7,)
(3,)
(4,)
(4,)
(12,)
(11,)
(4,)
(6,)
(2,)
(11,)
(6,)
(2,)
(5,)
(7,)
(3,)
(4,)
(13,)
(13,)
(13,)
(12,)
(8,)
(4,)
(5,)
(7,)
(8,)
(4,)
(5,)
(14,)
(8,)
(4,)
(6,)
(10,)
None
No product found
(2,)
(9,)
None
No product found
None
No product found
None
No product found
(6,)
(1,)
(5,)
(7,)
(3,)
(8,)
(14,)
(14,)
(5,)
(12,)
(6,)
(3,)
(7,)
(9,)
(6,)
(13,)
(15,)
(7,)
(14,)
(17,)
(9,)
(16,)
(13,)
(

In [61]:
# close cursor and connection
cursor.close()
connection.close()